In [1]:
import pandas as pd

fetal_health = pd.read_csv('fetal_health.csv')

fetal_health['histogram_tendency'] = fetal_health['histogram_tendency'].astype(str)
fetal_health = pd.get_dummies(fetal_health)
fetal_health.fetal_health = fetal_health.fetal_health.astype(int).astype(str) #make outcome categorical

In [2]:
from sklearn.model_selection import train_test_split

fetal_health, fetal_health_test = train_test_split(fetal_health, 
                                    test_size = .1) ## withold our test set


In [3]:
from utility_functions import * ## Import utility functions in the separate .py file

In [4]:
from sklearn.ensemble import RandomForestClassifier
fetal_health = rebalance_classes(fetal_health)
RandomForest_100 = RandomForestClassifier(n_estimators = 100)

rf_100_fit = RandomForest_100.fit(X = fetal_health.drop('fetal_health', axis = 1), y = fetal_health.fetal_health)
rf_100_preds = rf_100_fit.predict(X= fetal_health_test.drop('fetal_health', axis = 1))



In [5]:
## Process for KNN
from sklearn.preprocessing import StandardScaler

fetal_health_train = rebalance_classes(fetal_health)

scale = StandardScaler()

fetal_health_scaler = scale.fit(X = fetal_health.drop('fetal_health', axis = 1).to_numpy())

fetal_health_x = fetal_health_scaler.transform(X = fetal_health_train.drop('fetal_health', axis = 1).to_numpy())

fetal_health_test_x = fetal_health_scaler.transform(X = fetal_health_test.drop('fetal_health', axis = 1).to_numpy())



In [6]:
## fit knn model

from sklearn.neighbors import KNeighborsClassifier

knn_50 = KNeighborsClassifier(n_neighbors= 50)

knn_50_fit = knn_50.fit(X = fetal_health_x, y = fetal_health.fetal_health)

knn_50_preds = knn_50_fit.predict(X = fetal_health_test_x)

In [7]:
print("Accuracy for KNN")
print(print_metrics(accuracy_by_class(knn_50_preds, fetal_health_test.fetal_health)))

Overall accuracy: 0.8403755868544601
Accuracy for class 1: 0.8481012658227848
Accuracy for class 2: 0.8461538461538461
Accuracy for class 3: 0.75
Weighted Accuracy: 0.8147517039922102
Macro F1 score: 0.7530836281647678



In [8]:
print("Accuracy for Random Forest")
print(print_metrics(accuracy_by_class(rf_100_preds, fetal_health_test.fetal_health)))

Overall accuracy: 0.9624413145539906
Accuracy for class 1: 0.9810126582278481
Accuracy for class 2: 0.8717948717948718
Accuracy for class 3: 1.0
Weighted Accuracy: 0.9509358433409066
Macro F1 score: 0.9565265364669536

